## Bridge Course Grades Breakdown:
 - How did students do in Bridge? (Grade Distribution)
 - What was the gender breakdown of the Bridge courses?
 - What was the ethnic breakdown of the Bridge courses?
 - Average grades for B2C math and B2c ELA

TODO:
 - How many students graduated in 2016 that took B2C math and B2c ELA
 - Were there discipline differences between the groups?
 - What was the gender break-down of the non-bridge kids?
 - What was the ethnic break-down of the non-bridge kids?


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime
import csv
%matplotlib inline

source_path = 'D:\\PYTHON\\Bridge to College\\Data_Files\\' # Location of folder

source_grades = 'BtC_Cohort 1_Grades.csv'
source_assass = 'BtC_Cohort 1_Assess.csv'
source_enrollment = 'BtC_Cohort 1_Enrollment.csv' # Enrollment file name

output_report = 'Output_Report_Grades.csv' # Name of file that contains all to be used


In [ ]:
###############################################################
# Load Files and Create Data frames for Average Bridge grades #
###############################################################
start_time = time.time()
print('Loading: ' + source_grades)
tempdf = pd.read_csv(source_path + source_grades)
tempdf['StateCourseCode'].replace(to_replace=['02099'], value=2099, inplace=True)
tempdf['StateCourseCode'].replace(to_replace=['01069'], value=1069, inplace=True)
print(source_grades + ' loading completed: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))

##### Function to convert Letter grade to Number
def grade_letter_to_number(dataframe):    
    dataframe = dataframe[dataframe["LetterGrade"].isin(["P", "W", "NC", "S", "N"]) == False] 
    dataframe["grade_point"] = dataframe["LetterGrade"].map(lambda x: 4 if ("A" in x)
                                                              else 3.7 if ("A-" in x)
                                                              else 3.3 if ("B+" in x)
                                                              else 3 if ("B" in x)
                                                              else 2.7 if ("B-" in x)
                                                              else 2.3 if ("C+" in x)
                                                              else 2 if ("C" in x)
                                                              else 1.7 if ("C-" in x)
                                                              else 1.3 if ("D+" in x)
                                                              else 1 if ("D" in x)
                                                              else 0.7 if ("D-" in x)
                                                              else 0 if ("F" in x)
                                                              else x)
    
    return dataframe
#######################################################################################
##### Function to Convert Number grade to Letter grade
def grade_number_to_letter(dataframe):    
    dataframe = dataframe[dataframe['NumAvgGrade'].isin(['P', 'W', 'NC', 'S', 'N']) == False] 
    dataframe['AvgGrade'] = dataframe['NumAvgGrade'].map(lambda x: 'A' if (4 == x)
                                                              else 'A-' if ((4 > x) & (3.66 <= x))
                                                              else 'B+' if ((3.66 > x) & (3.33 <= x))
                                                              else 'B' if ((3.33 > x) & (3 <= x))
                                                              else 'B-' if ((3 > x) & (2.66 <= x))
                                                              else 'C+' if ((2.66 > x) & (2.33 <= x))
                                                              else 'C' if ((2.33 > x) & (2 <= x))
                                                              else 'C-' if ((2 > x) & (1.66 <= x))
                                                              else 'D+' if ((1.66 > x) & (1.33 <= x))
                                                              else 'D' if ((1.33 > x) & (1 <= x))
                                                              else 'F' if (1 > x)
                                                              else x)
    
    return dataframe
#######################################################################################
##### Create the Math Data Grades Frame
mgdf = tempdf[(tempdf['SchoolYear'] == 2016) & 
             (tempdf['StateCourseCode'] == 2099) & 
             (tempdf['GradeLevelWhenCourseTaken'] == 12)][['ResearchID', 'Term', 'LetterGrade']]
mgdf['LetterGrade'] = mgdf['LetterGrade'].str.upper() 

temp_mgdf = mgdf[mgdf['LetterGrade'].isin(['P', 'W', 'NC', 'S', 'N']) != False] 
temp_mgdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)

mgdf = grade_letter_to_number(mgdf)

for i in (mgdf['ResearchID']):
    (mgdf.loc[mgdf['ResearchID'] == i, 'NumAvgGrade']) = mgdf['grade_point'].loc[(mgdf['ResearchID'] == i)].mean(axis=0)

mgdf = grade_number_to_letter(mgdf)
mgdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
mgdf = mgdf[['ResearchID', 'AvgGrade']]
temp_mgdf = temp_mgdf.rename(columns = {'LetterGrade':'AvgGrade'})
temp_mgdf = temp_mgdf[temp_mgdf.columns.drop('Term')]

mgdf = pd.merge(mgdf, temp_mgdf, how='outer')
mgdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)

print('Math Grades Data Frame Completed: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))
#######################################################################################
##### Create the Enblish Grades Data Frame
egdf = tempdf[(tempdf['SchoolYear'] == 2016) & 
             (tempdf['StateCourseCode'] == 1069) & 
             (tempdf['GradeLevelWhenCourseTaken'] == 12)][['ResearchID', 'Term', 'LetterGrade']]
egdf['LetterGrade'] = egdf['LetterGrade'].str.upper() 

temp_egdf = egdf[egdf['LetterGrade'].isin(['P', 'W', 'NC', 'S', 'N']) != False] 
temp_egdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)

egdf = grade_letter_to_number(egdf)

for i in (egdf['ResearchID']):
    (egdf.loc[egdf['ResearchID'] == i, 'NumAvgGrade']) = egdf['grade_point'].loc[(egdf['ResearchID'] == i)].mean(axis=0)

egdf = grade_number_to_letter(egdf)
egdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
egdf = egdf[['ResearchID', 'AvgGrade']]
temp_egdf=temp_egdf.rename(columns = {'LetterGrade':'AvgGrade'})
temp_egdf = temp_egdf[temp_egdf.columns.drop('Term')]

egdf = pd.merge(egdf, temp_egdf, how='outer')
egdf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)

print('English Grades Data Frame Completed: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))
#######################################################################################
##### Add SBA levels to combined Data Frame
tempdf = pd.read_csv(source_path + source_assass)
print('Assess Data Frame Loaded: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))
#######################################################################################
madf = tempdf[(tempdf['Subject'] == 'Smarter Balanced Math') &
             (tempdf['TestGrade'] == 11) & (tempdf['SchoolYear'] == 2015) & 
             (tempdf['PerformanceLevel'] != '')][['ResearchID', 'PerformanceLevel', 'Subject', 'ScaleScore']]
madf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
math_df = pd.merge(mgdf, madf, how='outer')
math_df.fillna('', inplace=True)

eadf = tempdf[(tempdf['Subject'] == 'Smarter Balanced ELA') &
             (tempdf['TestGrade'] == 11) & (tempdf['SchoolYear'] == 2015) & 
             (tempdf['PerformanceLevel'] != '')][['ResearchID', 'PerformanceLevel', 'Subject', 'ScaleScore']]
eadf.drop_duplicates(subset='ResearchID', keep='last', inplace=True)
english_df = pd.merge(egdf, eadf, how='outer')
english_df.fillna('', inplace=True)

print('Combined Assess Data Frame Completed: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))
###########################################
#####     Loading Enrollment File     #####
###########################################
print('Loading: ' + source_enrollment)
tempdf = pd.read_csv(source_path + source_enrollment)

enroll_gender_df = tempdf[['ResearchID', 'Gender']]
enroll_race_df = tempdf[['ResearchID', 'FederalEthRaceRollupName']]
enroll_days_df = tempdf[(tempdf['SchoolYear'] == 2016) & 
                        (tempdf['ExpectedGradYear'] == 2016) & 
                        (tempdf['GradeLevel'] == 12)][['ResearchID', 'NumDaysPresent']]

english_df = pd.merge(english_df, enroll_gender_df, how = "left", on = "ResearchID")
english_df.drop_duplicates(subset=None, keep='first', inplace=True)
math_df = pd.merge(math_df, enroll_gender_df, how = "left", on = "ResearchID")
math_df.drop_duplicates(subset=None, keep='first', inplace=True)

english_df = pd.merge(english_df, enroll_race_df, how = "left", on = "ResearchID")
english_df.drop_duplicates(subset=None, keep='first', inplace=True)
math_df = pd.merge(math_df, enroll_race_df, how = "left", on = "ResearchID")
math_df.drop_duplicates(subset=None, keep='first', inplace=True)

english_df = pd.merge(english_df, enroll_days_df, how = "left", on = "ResearchID")
english_df.drop_duplicates(subset=None, keep='first', inplace=True)
math_df = pd.merge(math_df, enroll_days_df, how = "left", on = "ResearchID")
math_df.drop_duplicates(subset=None, keep='first', inplace=True)


print('Total Execution Time: ' + str((time.time() - start_time) / 60) + ' minutes.')



In [ ]:
#############################################
#####     Functions to write row data     #####
#############################################
def write_row (group, level, num_students, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W): 
    writer.writerow({'Student_Group': group, 
                     'Category': level, 
                     '# of students': num_students, 
                     'A':A,
                     'A-':Aminus,
                     'B+':Bplus,
                     'B':B,
                     'B-':Bminus,
                     'C+':Cplus,
                     'C':C,
                     'C-':Cminus,
                     'D+':Dplus,
                     'D':D,
                     'F':F,
                     'NC':NC,
                     'N':N,
                     'P':P,
                     'S':S,
                     'W':W}) 


#######################################################
#####            Write a simple row Row           #####
#######################################################
def write_simple_line (row_name):
    write_row(row_name, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '')
    
#######################################################
## Total Function to Generate Data to Write to Row ##
#######################################################
def write_total_row (row_name, data_frame, catagory_value):
    total = data_frame[data_frame['AvgGrade'] != '']['ResearchID'].nunique()
    A = data_frame[data_frame['AvgGrade'] == 'A']['ResearchID'].nunique()
    Aminus = data_frame[data_frame['AvgGrade'] == 'A-']['ResearchID'].nunique()
    Bplus = data_frame[data_frame['AvgGrade'] == 'B+']['ResearchID'].nunique()
    B = data_frame[data_frame['AvgGrade'] == 'B']['ResearchID'].nunique()
    Bminus = data_frame[data_frame['AvgGrade'] == 'B-']['ResearchID'].nunique()
    Cplus = data_frame[data_frame['AvgGrade'] == 'C+']['ResearchID'].nunique()
    C = data_frame[data_frame['AvgGrade'] == 'C']['ResearchID'].nunique()
    Cminus = data_frame[data_frame['AvgGrade'] == 'C-']['ResearchID'].nunique()
    Dplus = data_frame[data_frame['AvgGrade'] == 'D+']['ResearchID'].nunique()
    D = data_frame[data_frame['AvgGrade'] == 'D']['ResearchID'].nunique()
    F = data_frame[data_frame['AvgGrade'] == 'F']['ResearchID'].nunique()
    NC = data_frame[data_frame['AvgGrade'] == 'NC']['ResearchID'].nunique()
    N = data_frame[data_frame['AvgGrade'] == 'N']['ResearchID'].nunique()
    P = data_frame[data_frame['AvgGrade'] == 'P']['ResearchID'].nunique()
    S = data_frame[data_frame['AvgGrade'] == 'S']['ResearchID'].nunique()
    W = data_frame[data_frame['AvgGrade'] == 'W']['ResearchID'].nunique()

    write_row(row_name, catagory_value, total, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W)
    
#######################################################
## Generic Function to Generate Data to Write to Row ##
#######################################################
def write_catagory_row (row_name, data_frame, catagory, catagory_value):
    total = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] != '')]['ResearchID'].nunique()
    A = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'A')]['ResearchID'].nunique()
    Aminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'A-')]['ResearchID'].nunique()
    Bplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B+')]['ResearchID'].nunique()
    B = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B')]['ResearchID'].nunique()
    Bminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'B-')]['ResearchID'].nunique()
    Cplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C+')]['ResearchID'].nunique()
    C = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C')]['ResearchID'].nunique()
    Cminus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'C-')]['ResearchID'].nunique()
    Dplus = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'D+')]['ResearchID'].nunique()
    D = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'D')]['ResearchID'].nunique()
    F = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'F')]['ResearchID'].nunique()
    NC = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'NC')]['ResearchID'].nunique()
    N = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'N')]['ResearchID'].nunique()
    P = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'P')]['ResearchID'].nunique()
    S = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'S')]['ResearchID'].nunique()
    W = data_frame[(data_frame[catagory] == catagory_value) & (data_frame['AvgGrade'] == 'W')]['ResearchID'].nunique()

    write_row(row_name, catagory_value, total, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, F, NC, N, P, S, W)


In [ ]:
####################################################
#####     Set up and Create the Grades CSV     #####
####################################################

with open(source_path + output_report, "w", newline='') as csvfile:
    fieldnames = ['Student_Group', 'Category', '# of students', 
                  'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'F', 'NC', 'N', 'P', 'S', 'W']
                  
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    ##### Write English
    write_simple_line('Bridge English')
    write_total_row('English SBA Level', english_df, 'All English Bridge')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L1')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L2')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L3')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'L4')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', 'BA')
    write_catagory_row ('English SBA Level', english_df, 'PerformanceLevel', '')

    ##### Write English Gender
    write_catagory_row ('English Gender', english_df, 'Gender', 'F')
    write_catagory_row ('English Gender', english_df, 'Gender', 'M')

    ##### Write English Ethnic Race
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'White')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Two or More Races')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Black/African American')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Asian')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_catagory_row ('English Ethnic Race', english_df, 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')

     ##### Write Math Data
    write_simple_line('')
    write_simple_line('Bridge Math')
    write_total_row('Math SBA Level', math_df, 'All Math Bridge')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L1')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L2')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L3')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'L4')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', 'BA')
    write_catagory_row ('Math SBA Level', math_df, 'PerformanceLevel', '')

    ##### Write Math Gender
    write_catagory_row ('Math Gender', math_df, 'Gender', 'F')
    write_catagory_row ('Math Gender', math_df, 'Gender', 'M')

    ##### Write Math Ethnic Race
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'White')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Hispanic/Latino of any race(s)')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Two or More Races')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Black/African American')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Asian')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'Native Hawaiian/Other Pacific Islander')
    write_catagory_row ('Math Ethnic Race', math_df, 'FederalEthRaceRollupName', 'American Indian/Alaskan Native')

csvfile.close()
print('Execution Completed: ' + str(datetime.datetime.now().strftime('%Y %m %d,  %H:%M:%S')))
